# 课时41 TensorFlow2.0版本GAN

In [42]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
tf.__version__

'2.1.0'

## 1. 导入数据集

In [43]:
(train_images, train_labels),(_, _) = tf.keras.datasets.mnist.load_data()

In [44]:
# 查看数据的shape
train_images.shape

(60000, 28, 28)

In [45]:
# 查看数据的类型
train_images.dtype

dtype('uint8')

In [47]:
# 数据归一化
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5      # Normalize the images to [-1, 1]

In [48]:
# 将数据转换为dataset
BATCH_SIZE = 256
BUFFER_SIZE = 60000
datasets = tf.data.Dataset.from_tensor_slices(train_images)

In [49]:
datasets = datasets.shuffle(BUFFER_SIZE).batch(batch_size=BATCH_SIZE)

In [50]:
datasets

<BatchDataset shapes: (None, 28, 28, 1), types: tf.float32>

# 2. 创建生成器和判别器

In [65]:
def generator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=256, input_shape=(100,), use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dense(units=512, use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dense(units=28*28*1, use_bias=False, activation='tanh'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Reshape((28, 28, 1))])
    return model

In [52]:
def discriminator_model():
    model = tf.keras.Sequential([
        # 由于输入到判别器的是图片，而这里展示的是使用全连接神经网络的示例，因此需要将输入的图片Flatten
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=512, use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        
        tf.keras.layers.Dense(units=256, use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        
        tf.keras.layers.Dense(units=1)])
    return model

## 3. 定义loss函数和优化器

In [53]:
# 定义一个可调用的Loss函数，方便后面进行调用
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [54]:
def discriminator_loss(real_image, fake_image):
    real_loss = cross_entropy(tf.ones_like(real_image), real_image)
    fake_loss = cross_entropy(tf.zeros_like(fake_image), fake_image)
    return real_loss + fake_loss

In [55]:
def generator_loss(fake_image):
    return cross_entropy(tf.ones_like(fake_image), fake_image)

In [56]:
generator_opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
discriminator_opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [57]:
EPOCHS = 100
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [58]:
generator = generator_model()
discriminator = discriminator_model()

In [63]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_opt.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_opt.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [60]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i, :, :, 0] + 1)/2, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [61]:
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
            print('.', end='')
        print()

        generate_and_save_images(generator,
                             epoch + 1,
                             seed)


    generate_and_save_images(generator,
                           epochs,
                           seed)

In [64]:
train(datasets, EPOCHS)

..........................................................

KeyboardInterrupt: 